In [1]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression

In [2]:
N = 1000000
n = 1000
m = 100
Wreal = torch.cuda.FloatTensor(n,m).normal_()
X = torch.cuda.FloatTensor(N,n).normal_()
Y = torch.argmax(X@Wreal+5*torch.randn(N,m).cuda(),dim=1).cuda()

In [3]:
W = Variable(torch.cuda.FloatTensor(n,m).zero_(),requires_grad=True)
b = Variable(torch.cuda.FloatTensor(m).zero_(),requires_grad=True)
def model(X):
    return F.log_softmax(X@W+b)

In [4]:
optim = torch.optim.LBFGS([W,b],lr=1,max_iter=100,tolerance_grad=1e-3)
def closure():
    optim.zero_grad()
    loss = torch.nn.NLLLoss()(model(X),Y) + N*1e-9*torch.sum(W**2)
    loss.backward()
    print (loss.item())
    return loss

In [5]:
%%time
optim.step(closure)

/home/shane/miniconda3/envs/lstsq/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.


4.604015350341797
4.603674411773682
1.952467441558838
2.133491277694702
1.8471050262451172
1.8384475708007812
1.8242239952087402
1.8231489658355713
1.8227170705795288
1.8225266933441162
1.822489857673645
1.8224807977676392
1.822475552558899
1.8224586248397827
1.8224490880966187
1.8224576711654663
1.8224436044692993
1.8224432468414307
1.8224436044692993
1.8224437236785889
1.8224437236785889
CPU times: user 2.67 s, sys: 922 ms, total: 3.59 s
Wall time: 3.58 s


tensor(4.6040, device='cuda:0', grad_fn=<ThAddBackward>)